# PawCore Phase 1: Initial Problem Analysis

This notebook uses AISQL functions to analyze our Q4 2024 performance issues:
1. AI_EXTRACT: Analyze financial reports
2. AI_SENTIMENT: Analyze customer reviews
3. AI_SUMMARIZE_AGG: Analyze Slack communications


In [ ]:
-- Set up our environment
USE ROLE accountadmin;
USE WAREHOUSE PAWCORE_DEMO_WH;
USE DATABASE PAWCORE_ANALYTICS;
USE SCHEMA UNSTRUCTURED;

-- Verify our data is loaded
SELECT COUNT(*) as doc_count FROM PARSED_DOCUMENTS
WHERE file_name LIKE '%Q4_2024_PawCore_Financial_Report.md%';


In [ ]:
-- Extract key financial metrics from Q4 report
SELECT
    AI_EXTRACT(
        content,
        'Extract the following metrics from this financial report:
        1. Total revenue and forecast
        2. Revenue by region
        3. Revenue by product
        4. Key performance issues'
    ) as financial_analysis
FROM UNSTRUCTURED.PARSED_DOCUMENTS
WHERE file_name LIKE '%Q4_2024_PawCore_Financial_Report.md%';


In [ ]:
-- Analyze Q4 2024 customer reviews with focus on SmartCollar and EMEA
WITH review_data AS (
    SELECT 
        REVIEW_TEXT,
        PRODUCT,
        REGION,
        DATE
    FROM @PAWCORE_DATA_STAGE/documents/customer_reviews.csv
    (FILE_FORMAT => CSV_FORMAT)
    WHERE DATE >= '2024-10-01' 
    AND DATE <= '2024-12-31'
    AND PRODUCT = 'SmartCollar'
    AND REGION = 'EMEA'
)
SELECT
    AI_SENTIMENT(REVIEW_TEXT) as sentiment_score,
    AI_EXTRACT(REVIEW_TEXT, 'What specific issues are mentioned in this review?') as issues,
    DATE,
    REVIEW_TEXT
FROM review_data
ORDER BY DATE;


In [ ]:
-- Analyze Slack messages for Q4 2024 focusing on SmartCollar issues
WITH slack_data AS (
    SELECT 
        TEXT,
        SLACK_CHANNEL,
        TO_TIMESTAMP(THREAD_ID) as timestamp
    FROM @PAWCORE_DATA_STAGE/documents/pawcore_slack.csv
    (FILE_FORMAT => CSV_FORMAT)
    WHERE TO_TIMESTAMP(THREAD_ID) >= '2024-10-01'
    AND TO_TIMESTAMP(THREAD_ID) <= '2024-12-31'
    AND (LOWER(TEXT) LIKE '%smartcollar%' OR LOWER(TEXT) LIKE '%lot 341%')
)
SELECT
    AI_SUMMARIZE_AGG(
        TEXT,
        'Summarize the key events and timeline regarding SmartCollar issues',
        'timestamp'
    ) as timeline_summary,
    SLACK_CHANNEL,
    COUNT(*) as message_count
FROM slack_data
GROUP BY SLACK_CHANNEL;
